In [342]:
import numpy as np
import scipy.optimize as opt
np.set_printoptions(suppress=True)



In [343]:

def constant_sum(p: np.array):
    p = np.array(p)
    def f(x: np.array):
        x = np.array(x)
        return p @ x
    
    def f_grad(x: np.array):
        x = np.array(x)
        return p
    
    return f, f_grad

def constant_geo_mean():
    def f(x: np.array):
        x = np.array(x)
        return np.prod(x) ** (1 / len(x))
    
    def f_grad(x: np.array):
        x = np.array(x)
        return f(x) / (x * len(x))
    
    return f, f_grad

def stableswap(p: np.array, X: float):
    p = np.array(p)
    def f(x: np.array):
        x = np.array(x)
        return np.prod(x) ** (1 / len(x)) + X * np.sum(p * x)
    
    def f_grad(x: np.array):
        x = np.array(x)
        return (np.prod(x) ** (1 / len(x))) / (x * len(x)) + X * p
    
    return f, f_grad

AMMs = [
    # constant_sum([1, 5]),
    constant_sum([10, 20, 40]),
    stableswap([1, 2, 4], 1),
    constant_geo_mean(),
    # constant_geo_mean(),
    # constant_geo_mean()
]

N = len(AMMs)
K = 3

AMM_funs, AMM_grads = zip(*AMMs)

constraints = []

Delta = np.zeros((N, K))
initial_quantities = np.array([
    [10, 10, 10],
    [10, 20, 5],
    [10, 10, 10],
    # [20, 10, 1],
    # [10, 25, 100]
])

# initial prices, normalized
initial_prices = np.array([
    grad(q) / np.sum(grad(q)) 
    for q, grad 
    in zip(initial_quantities, AMM_grads)])

# initial values of trading functions
orig_ks = np.array([f(q) for f, q in zip(AMM_funs, initial_quantities)])
# amount to scale profit, in terms of increase in trading function
multipliers = np.sum(initial_prices * initial_quantities, axis=1)


# # trading function must increase (don't need this bc they increase due to objective)
# def objective_satisfied(d, f, i):
#     d = np.reshape(d, (N, K))
#     return f(initial_quantities[i] + d[i]) - f(initial_quantities[i])

# for i, f in enumerate(AMM_funs):
#     constraints.append({
#         'type': 'ineq',
#         'fun': lambda d, f=f, i=i: objective_satisfied(d, f, i)
#     })

# reserves must be nonnegative
def nonnegative(d):
    d = np.reshape(d, (N, K))
    val = initial_quantities + d
    return np.min(val)


constraints.append({
    'type': 'ineq',
    'fun': lambda d: nonnegative(d)
})

# preserve network flow (each asset has non-positive sum)
# this is an inequality instead of equality to preserve convexity
def sums_to_zero(d):
    d = np.reshape(d, (N, K))
    return -np.sum(d, axis=0)
constraints.append({
    'type': 'ineq',
    'fun': lambda d: sums_to_zero(d)
})

# profit = proportional change in k times multiplier
def profits(d):
    new_quantities = initial_quantities + d
    new_ks = np.array([f(q) for f, q in zip(AMM_funs, new_quantities)])
    profit_per_amm = multipliers * (new_ks / orig_ks - 1)
    return profit_per_amm

for i in range(1, N):
    constraints.append({
        'type': 'ineq',
        'fun': lambda d, i=i: profits(d.reshape((N, K)))[i] - profits(d.reshape((N, K)))[0]
    })


def objective(d):
    d = np.reshape(d, (N, K))
    # only optimize for the first AMM since the others are constrained to move with it
    return -profits(d)[0]
    # return -np.sum(profits(d))

res = opt.minimize(objective, Delta.flatten(), constraints=constraints)

d = res.x.reshape((N, K))
new_q = initial_quantities + d
new_ks = np.array([amm_f(q) for amm_f, q in zip(AMM_funs, new_q)])
new_ks / orig_ks
print(f"original prices: \n{initial_prices}")
print(f"multipliers: \n{multipliers}")
print(f"original quantities: \n{initial_quantities}")
print(f"new quantities: \n{new_q}")
print(f"delta: \n{d}")
print(f"change in k: \n{new_ks / orig_ks}")
prices = np.array([grad(q) / np.sum(grad(q)) for q, grad in zip(new_q, AMM_grads)])
print(f"new prices: \n{prices}")
print(f"profits: \n{profits(d)}")
print(res)



original prices: 
[[0.14285714 0.28571429 0.57142857]
 [0.16326531 0.26530612 0.57142857]
 [0.33333333 0.33333333 0.33333333]]
multipliers: 
[10.          9.79591837 10.        ]
original quantities: 
[[10 10 10]
 [10 20  5]
 [10 10 10]]
new quantities: 
[[-0.         13.39652439 11.69795147]
 [11.23984044 15.47499156  7.73790078]
 [18.76015956 11.12848405  5.56414775]]
delta: 
[[-10.           3.39652439   1.69795147]
 [  1.23984044  -4.52500844   2.73790078]
 [  8.76015956   1.12848405  -4.43585225]]
change in k: 
[1.05121221 1.05227911 1.05121221]
new prices: 
[[0.14285714 0.28571429 0.57142857]
 [0.16508569 0.2783058  0.55660851]
 [0.16508753 0.27830101 0.55661146]]
profits: 
[0.51212209 0.51212194 0.51212209]
 message: Optimization terminated successfully
 success: True
  status: 0
     fun: -0.5121220944633097
       x: [-1.000e+01  3.397e+00  1.698e+00  1.240e+00 -4.525e+00
            2.738e+00  8.760e+00  1.128e+00 -4.436e+00]
     nit: 37
     jac: [-1.429e-01 -2.857e-01 -5.7